In [1]:
import sys
import os
import numpy as np
import pandas as pd
import FdsPy.spar.myspengine as sp
import FdsPy.qe.myqengine as qe

In [2]:
time_series = qe.TimeSeries(start_date = '0M',end_date = '0M')

univ = qe.ScreenUniverse(universe_expr = '(ISAV(FFD_SHR_CLS_AUM(0,ACT,ROLL,USD))=1 AND ISAV(P_PRICE(0,USD))=1  AND UPPERCASE(P_EXCOUNTRY(ISO2))=UPPERCASE("US") AND PROPER(FFD_CLASS_ASSET(TEXT))=PROPER("EQUITY"))=1')
data_items = {'name':"FFD_NAME",
              'price':'P_PRICE(0,USD)',
              'aum':"FFD_SHR_CLS_AUM(0,ACT,ROLL,USD)",
              'lss_ticker':"LSS_TICKER",
              'ffd_bmk_id':"FFD_BMK_ID",
              'ffd_brand':"FFD_BRAND",
              'focus':'FFD_CLASS_FOCUS(TEXT)',
              'segment':'FFD_SEG',
              'niche':'FFD_CLASS_NICHE(TEXT)',
              'class':'FFD_CLASS_ASSET(TEXT)',
              'category':'FFD_CLASS_CAT(TEXT)'
              }

req = qe.Calculation(universe = univ, dates = time_series,data_dict= data_items)
req.query()


,DATE,UNIVERSE,name,price,aum,lss_ticker,ffd_bmk_id,ffd_brand,focus,segment,niche,class,category
0,20230831,RMT,Royce Micro-Cap Trust Incorporated,8.98,5.010069e+08,RMT-US,R.2000,Royce,Micro Cap,Equity: U.S. - Micro Cap,Broad-based,Equity,Size and Style
1,20230831,FUNYX,Pioneer Fundamental Growth Fund,30.59,3.814054e+09,None,R.1000G,Pioneer,Large Cap,Equity: U.S. - Large Cap Growth,Growth,Equity,Size and Style
2,20230831,ABCAX,American Aadvantage Funds,20.69,1.386211e+08,None,R.1000V,American Beacon,Total Market,Equity: U.S. - Total Market,Broad-based,Equity,Size and Style
3,20230831,ADX,Adams Diversified Equity Fund Incorporated,17.63,2.471080e+09,ADX-US,NOINDEX,Adams,Large Cap,Equity: U.S. - Large Cap,Broad-based,Equity,Size and Style
4,20230831,EEA,European Equity Fund Incorporated,8.32,6.924492e+07,EEA-US,990500,DWS,Total Market,Equity: Developed Europe - Total Market,Broad-based,Equity,Size and Style


In [50]:
df = req.data
df.head()

,DATE,UNIVERSE,name,price,aum,lss_ticker,ffd_bmk_id,ffd_brand,focus,segment,niche,class,category
0,20230831,RMT,Royce Micro-Cap Trust Incorporated,8.98,5.010069e+08,RMT-US,R.2000,Royce,Micro Cap,Equity: U.S. - Micro Cap,Broad-based,Equity,Size and Style
1,20230831,FUNYX,Pioneer Fundamental Growth Fund,30.59,3.814054e+09,None,R.1000G,Pioneer,Large Cap,Equity: U.S. - Large Cap Growth,Growth,Equity,Size and Style
2,20230831,ABCAX,American Aadvantage Funds,20.69,1.386211e+08,None,R.1000V,American Beacon,Total Market,Equity: U.S. - Total Market,Broad-based,Equity,Size and Style
3,20230831,ADX,Adams Diversified Equity Fund Incorporated,17.63,2.471080e+09,ADX-US,NOINDEX,Adams,Large Cap,Equity: U.S. - Large Cap,Broad-based,Equity,Size and Style
4,20230831,EEA,European Equity Fund Incorporated,8.32,6.924492e+07,EEA-US,990500,DWS,Total Market,Equity: Developed Europe - Total Market,Broad-based,Equity,Size and Style


In [51]:
df=df.dropna().sort_values('aum',ascending = False).head(100)

In [4]:
import pandas as pd

#Spar inputs
doc_name = "PERSONAL:RETURNS_API"
comp_name = 'Cumulative Returns Table'
comp_cat = 'Raw Data / Returns'
bench_prefix = 'SPN'

ports = df['lss_ticker'].to_list()
bench = 'SP50'
ret_type = 'NTR'
freq = 'Daily'
curr = 'USD'
startdate = '20230101'
enddate = '20230901'

#post call to SPAR api
spar_df = sp.calc_spar_data(doc_name,comp_name,comp_cat,ports,bench,bench_prefix,ret_type,startdate,enddate,freq,curr)

In [36]:
df_full = pd.DataFrame()
for port in list(spar_df.keys())[1:]:
    df_temp = spar_df[port][0]
    df_temp['ticker'] = port
    df_temp.iloc[:,1] = df_temp.iloc[:,1]
    df_temp =df_temp.rename(columns = {df_temp.columns[1]:port})
    df_temp= df_temp.set_index(['Dates'])
    df_full= pd.concat([df_full,df_temp[port]],axis=1)
df_full


,SWPPX-US,JLGMX-US,DFQTX-US,DFEOX-US,DFIEX-US,DFCEX-US,TEQWX-US,OIEJX-US,TRJZX-US,TRZAX-US,...,BGRIX-US,JENIX-US,JOHIX-US,HGOFX-US,GTEYX-US,TRQZX-US,MIDLX-US,DURPX-US,FTHSX-US,TRMIX-US
12/30/2022,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
01/02/2023,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
01/03/2023,-0.391421,-0.682008,-0.405161,-0.436386,0.370084,0.446885,-0.405141,-0.221535,-0.370670,0.000000,...,0.055870,0.036584,-0.349839,-0.239587,-0.140671,-0.179217,-0.068594,-0.291418,-0.173571,-0.141357
01/04/2023,0.357390,-0.277068,0.478823,0.469966,1.480382,2.284009,0.441996,0.531689,0.032233,0.806034,...,1.475023,0.493874,0.749623,0.419295,0.196961,1.433690,0.411663,0.524460,0.491751,1.978794
01/05/2023,-0.799863,-1.364025,-0.405161,-0.402805,0.666179,2.333659,-0.736638,-0.310154,-1.756651,-0.025993,...,-0.391106,-1.060908,-0.049983,-1.197964,-0.365766,0.156809,-0.240117,-0.291418,-0.057862,1.272083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
08/28/2023,16.695034,22.570333,12.583713,13.547624,8.775996,8.788756,15.911595,-0.159601,33.714753,5.278210,...,9.338480,10.599755,10.294866,27.343526,11.193490,11.693545,6.003443,13.204534,8.562328,8.975252
08/29/2023,18.396873,25.127875,14.030886,15.103540,10.432085,9.994149,17.642725,0.562762,36.196614,6.084244,...,10.903012,12.015340,12.093969,29.979042,12.126712,13.216849,7.478572,14.615161,9.921887,10.176666
08/30/2023,18.873386,25.895137,14.439065,15.509429,10.432085,9.943925,18.121543,0.832858,37.002421,6.318264,...,11.651754,12.401411,11.694147,30.907458,12.409497,13.922485,7.444263,14.909055,10.153305,10.706713
08/31/2023,18.703201,26.449273,14.290641,15.407957,10.130957,8.939430,18.011044,0.382708,37.179691,5.824230,...,11.428248,11.776347,11.644164,31.805942,12.409497,13.776880,7.615785,14.791528,10.297945,10.565356


In [52]:
df_m = pd.DataFrame(df_full.stack().rename('cum_ret')).reset_index()
df_m = df_m.rename(columns = {'level_0':'DATE',
                              'level_1':'lss_ticker'})
df_m = df_m.set_index(['DATE','lss_ticker'])

In [53]:
df = df.set_index(['lss_ticker'])

In [54]:
df_m = df_m.join(df,how='left')
df_m

cum_ret      DATE UNIVERSE  \
DATE       lss_ticker                                 
12/30/2022 SWPPX-US     0.000000  20230831    SWPPX   
           JLGMX-US     0.000000  20230831    JLGMX   
           DFQTX-US     0.000000  20230831    DFQTX   
           DFEOX-US     0.000000  20230831    DFEOX   
           DFIEX-US     0.000000  20230831    DFIEX   
...                          ...       ...      ...   
09/01/2023 TRQZX-US    14.381718  20230831    TRQZX   
           MIDLX-US     7.512881  20230831    MIDLX   
           DURPX-US    15.144159  20230831    DURPX   
           FTHSX-US    11.455018  20230831    FTHSX   
           TRMIX-US    11.130728  20230831    TRMIX   

                                                               name   price  \
DATE       lss_ticker                                                         
12/30/2022 SWPPX-US                            Schwab Capital Trust   69.75   
           JLGMX-US                                JPMorgan Trust I   59.33   
           DFQTX-US                  DFA US Core Equity 2 Portfolio   30.80   
           DFEOX-US                  DFA US Core Equity 1 Portfolio   34.12   
           DFIEX-US         DFA International Core Equity Portfolio   14.63   
...                                                             ...     ...   
09/01/2023 TRQZX-US            Rowe T Price Mid Capital Growth Fund  101.58   
           MIDLX-US                              MFS Series Trust V   31.37   
           DURPX-US    DFA US High Relative Profitability Portfolio   19.53   
           FTHSX-US                            Capitol Series Trust   38.13   
           TRMIX-US                 Rowe T Price Mid-Cap Value Fund   31.29   

                                aum ffd_bmk_id        ffd_brand  \
DATE       lss_ticker                                             
12/30/2022 SWPPX-US    7.462342e+10       SP50           Schwab   
           JLGMX-US    4.126916e+10    R.1000G         JPMorgan   
           DFQTX-US    3.097797e+10     R.3000              DFA   
           DFEOX-US    3.047367e+10     R.3000              DFA   
           DFIEX-US    2.985577e+10     991000              DFA   
...                             ...        ...              ...   
09/01/2023 TRQZX-US    5.082423e+09     R.MIDG    T. Rowe Price   
           MIDLX-US    4.841225e+09   MS661687              MFS   
           DURPX-US    4.792612e+09     R.1000              DFA   
           FTHSX-US    4.779479e+09     R.2000  Fuller & Thaler   
           TRMIX-US    4.693800e+09     R.MIDV    T. Rowe Price   

                                 focus  \
DATE       lss_ticker                    
12/30/2022 SWPPX-US          Large Cap   
           JLGMX-US          Large Cap   
           DFQTX-US       Total Market   
           DFEOX-US       Total Market   
           DFIEX-US       Total Market   
...                                ...   
09/01/2023 TRQZX-US            Mid Cap   
           MIDLX-US    Extended Market   
           DURPX-US          Large Cap   
           FTHSX-US          Small Cap   
           TRMIX-US            Mid Cap   

                                                       segment        niche  \
DATE       lss_ticker                                                         
12/30/2022 SWPPX-US                 Equity: U.S.  -  Large Cap  Broad-based   
           JLGMX-US         Equity: Global  - Large Cap Growth       Growth   
           DFQTX-US              Equity: U.S.  -  Total Market  Broad-based   
           DFEOX-US              Equity: U.S.  -  Total Market  Broad-based   
           DFIEX-US    Equity: Global Ex-U.S.  -  Total Market  Broad-based   
...                                                        ...          ...   
09/01/2023 TRQZX-US              Equity: U.S. - Mid Cap Growth       Growth   
           MIDLX-US         Equity: Global  - Extended Market   Broad-based   
           DURPX-US                 Equity: U.S.  -  Large Cap  Broad-ba

In [56]:
df_m.to_pickle('../../data/Performance Leaders/univ_cumulative_returns_100_20230915.pkl')